In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from sklearn.preprocessing import MinMaxScaler
import warnings 
from tqdm import tqdm
import os
import pickle
warnings.filterwarnings('ignore')

In [2]:
grid_be = gpd.read_file('./data_pe/grids_with_BE.geojson')

In [3]:
selectedColumns = ['fnid','pre_home_distance', 'post_home_distance', 'LU_Business',
                        'LU_City_Road', 'LU_Consumption', 'LU_Culture', 'LU_Industry',
                        'LU_Medical', 'LU_Park_&_Scenery', 'LU_Public', 'LU_Residence',
                        'LU_Science_&_Education', 'LU_Special', 'LU_Transportation', 'LU_Wild']
be_data = grid_be[selectedColumns]

In [4]:
grid_coords = grid_be.geometry.centroid
be_data['x'] = grid_coords.x
be_data['y'] = grid_coords.y

In [5]:
after_be_data = be_data.drop(columns=['pre_home_distance']).rename(columns={'post_home_distance': 'home_distance'})
before_be_data = be_data.drop(columns=['post_home_distance']).rename(columns={'pre_home_distance': 'home_distance'})

# load model

In [6]:
import SCBIRL_Global_PE.SCBIRLTransformer as SIRLT
import SCBIRL_Global_PE.utils as SIRLU
import SCBIRL_Global_PE.migrationProcess as SIRLP

data_dir = './data/'
before_migration_path = data_dir + 'before_migrt.json'
full_trajectory_path = data_dir + 'all_traj.json'

inputs, targets_action, pe_code, action_dim, state_dim = SIRLU.loadTrajChain(before_migration_path, full_trajectory_path)
model = SIRLT.avril(inputs, targets_action, pe_code, state_dim, action_dim, state_only=True)

In [7]:
def getComputeFunction(model, attribute_type):
    """Return the appropriate function to compute either 'value' or 'reward'."""
    if attribute_type == 'value':
        return lambda state,grid_code: np.max(model.QValue(state,grid_code))
    elif attribute_type == 'reward':
        return lambda state,grid_code: model.reward(state,grid_code)[0][0][0]
    else:
        raise ValueError("attribute_type should be either 'value' or 'reward'.")
    
Q = np.load('./data_pe/Q_matrix.npy')
with open('./data_pe/random_angle_list.pkl', 'rb') as file:
    angle_list = pickle.load(file)
def globalPE(coords,dimension):
    x,y = coords
    for k in range(1,dimension+1):
        theta = 2 * np.pi / 3  
        angle = angle_list[k-1]
        R = np.array([[np.cos(theta), -np.sin(theta)], [np.sin(theta), np.cos(theta)]])
        scale_factor = (200**(k/dimension))
        omega_n0 = np.array([np.cos(angle), np.sin(angle)]) * scale_factor
        omega_n1 = R.dot(omega_n0)
        omega_n2 = R.dot(omega_n1)

        coords = np.vstack((x, y))
        eiw0x = np.exp(1j * np.dot(omega_n0,coords))
        eiw1x = np.exp(1j * np.dot(omega_n1,coords))
        eiw2x = np.exp(1j * np.dot(omega_n2,coords))

        g_n = Q.dot(np.array([eiw0x, eiw1x, eiw2x]))
        if k == 1:
            g = g_n
        else:
            g = np.concatenate((g, g_n), axis=0)
    return g

# Calculate reward for all grids

In [8]:
model.loadParams('./model/params_transformer_pe.pickle')
rewardFunction = getComputeFunction(model, 'reward')
def calculate_reward(row):
    coords = (row.x, row.y)
    state = row[1:15]
    pe_code = globalPE(coords,len(state)).flatten()
    pe_code = np.expand_dims(np.expand_dims(np.expand_dims(pe_code, axis=0), axis=0), axis = 0)
    state = np.expand_dims(np.expand_dims(np.expand_dims(state, axis=0), axis=0), axis = 0)
    return float(rewardFunction(state, pe_code))
before_be_data['reward'] = before_be_data.apply(calculate_reward, axis=1)
before_be_data.to_csv('./data_pe/all_grid_before.csv')

load params from ./model/params_transformer_pe.pickle!


In [9]:
model_dir = './data_pe/after_migrt/model_random/'
save_dir = './data_pe/after_migrt/all_grid/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

pickle_files = [file for file in os.listdir(model_dir) if file.endswith('.pickle')]
for pickle_file in tqdm(pickle_files):
    temp_after_be_data = after_be_data.copy()
    file_path = os.path.join(model_dir,pickle_file)
 
    model.loadParams(file_path)
    rewardFunction = lambda state,grid_code: model.reward(state,grid_code)[0][0][0]
    rewards = []

    for index, row in temp_after_be_data.iterrows():
        coords = (row.x, row.y)
        state = row[1:15] 
        pe_code = globalPE(coords, len(state)).flatten()
        pe_code = np.expand_dims(np.expand_dims(np.expand_dims(pe_code, axis=0), axis=0), axis=0)
        state = np.expand_dims(np.expand_dims(np.expand_dims(state, axis=0), axis=0), axis=0)
        reward = float(rewardFunction(state, pe_code))
        rewards.append(reward)
    temp_after_be_data['reward'] = rewards
    
    # Save the updated DataFrame to a CSV file in the save_dir
    csv_file_name = pickle_file.replace('.pickle', '.csv')
    csv_file_path = os.path.join(save_dir, csv_file_name)
    temp_after_be_data.to_csv(csv_file_path, index=False)

  0%|          | 0/13 [00:00<?, ?it/s]

load params from ./data_pe/after_migrt/model_random/20190707.pickle!


  8%|▊         | 1/13 [07:53<1:34:36, 473.07s/it]

load params from ./data_pe/after_migrt/model_random/20190715.pickle!


 15%|█▌        | 2/13 [15:46<1:26:47, 473.39s/it]

...

 85%|████████▍ | 11/13 [1:27:09<15:53, 476.52s/it]

load params from ./data_pe/after_migrt/model_random/20191216.pickle!


 92%|█████████▏| 12/13 [1:35:02<07:55, 475.58s/it]

load params from ./data_pe/after_migrt/model_random/20191231.pickle!


100%|██████████| 13/13 [1:42:57<00:00, 475.19s/it]
